In [1]:
import pandas as pd
import numpy as np
import sqlite3

import os

#First make sure if the database is already there that we erase it prior to this lesson
if os.path.exists("Students.db"):
    os.remove("Students.db")

#Create the data
student_ids = list(range(1,101))
students = pd.DataFrame(student_ids)
students.columns = ['StudentID']
np.random.seed(0)
students['Classroom1'] = np.random.choice([True,False], 100, p=[.5,.5])
students['Classroom2'] = np.random.choice([True,False], 100, p=[.5,.5])
students['Grade1'] = (np.random.rand(100) * 50 + 50).astype(int)
students['Grade2'] = (np.random.rand(100) * 50 + 50).astype(int)
students['Absent1'] = (np.random.rand(100) * 5).astype(int)
students['Absent2'] = (np.random.rand(100) * 5).astype(int)
students = students.set_index("StudentID")
students1 = students[students['Classroom1']][['Grade1', 'Absent1']]
students2 = students[students['Classroom2']][['Grade2', 'Absent2']]

print(students1.head(5))
print()
print()
print(students2.head(5))
conn = sqlite3.connect('Students.db')
students1.to_sql("Class1", conn)
students2.to_sql("Class2", conn)

           Grade1  Absent1
StudentID                 
5              51        4
7              83        1
10             94        0
15             51        1
16             87        1


           Grade2  Absent2
StudentID                 
2              88        1
5              70        1
9              86        1
11             88        0
14             54        3


In [2]:
#A basic left join will find the matching records from the first table and then add in the columns from the right table
#Notice we have Class1 and Class2 before fields to denote which we are talking about
query = "SELECT * FROM Class1 LEFT JOIN Class2 ON Class1.StudentID = Class2.StudentID;"
df = pd.read_sql(query, conn)
print(df)

    StudentID  Grade1  Absent1  StudentID  Grade2  Absent2
0           5      51        4        5.0    70.0      1.0
1           7      83        1        NaN     NaN      NaN
2          10      94        0        NaN     NaN      NaN
3          15      51        1        NaN     NaN      NaN
4          16      87        1       16.0    62.0      3.0
5          17      66        0        NaN     NaN      NaN
6          23      63        2        NaN     NaN      NaN
7          25      59        3       25.0    63.0      0.0
8          27      84        4       27.0    84.0      2.0
9          30      86        1       30.0    68.0      2.0
10         31      62        1        NaN     NaN      NaN
11         33      75        1       33.0    52.0      3.0
12         35      60        2        NaN     NaN      NaN
13         41      93        1        NaN     NaN      NaN
14         42      55        4       42.0    59.0      3.0
15         44      56        0        NaN     NaN      N

In [3]:
#We can also pick specific columns from each table
query = """SELECT Class1.StudentID,
    Class1.Grade1,
    Class2.Grade2 FROM Class1
    LEFT JOIN Class2 ON Class1.StudentID = Class2.StudentID;"""
df = pd.read_sql(query, conn)
print(df)

    StudentID  Grade1  Grade2
0           5      51    70.0
1           7      83     NaN
2          10      94     NaN
3          15      51     NaN
4          16      87    62.0
5          17      66     NaN
6          23      63     NaN
7          25      59    63.0
8          27      84    84.0
9          30      86    68.0
10         31      62     NaN
11         33      75    52.0
12         35      60     NaN
13         41      93     NaN
14         42      55    59.0
15         44      56     NaN
16         47      78    75.0
17         48      59     NaN
18         49      57     NaN
19         50      74     NaN
20         52      97     NaN
21         54      87     NaN
22         55      95    76.0
23         56      54     NaN
24         58      79     NaN
25         59      98    69.0
26         60      64    97.0
27         61      62     NaN
28         62      55    95.0
29         64      96     NaN
30         65      83     NaN
31         66      89     NaN
32        

In [4]:
#As well, we are able to label the columns differently with AS
query = """SELECT Class1.StudentID AS ID,
    Class1.Grade1 AS G1,
    Class2.Grade2 AS G2 FROM Class1
    LEFT JOIN Class2 ON Class1.StudentID = Class2.StudentID;"""
df = pd.read_sql(query, conn)
print(df)

     ID  G1    G2
0     5  51  70.0
1     7  83   NaN
2    10  94   NaN
3    15  51   NaN
4    16  87  62.0
5    17  66   NaN
6    23  63   NaN
7    25  59  63.0
8    27  84  84.0
9    30  86  68.0
10   31  62   NaN
11   33  75  52.0
12   35  60   NaN
13   41  93   NaN
14   42  55  59.0
15   44  56   NaN
16   47  78  75.0
17   48  59   NaN
18   49  57   NaN
19   50  74   NaN
20   52  97   NaN
21   54  87   NaN
22   55  95  76.0
23   56  54   NaN
24   58  79   NaN
25   59  98  69.0
26   60  64  97.0
27   61  62   NaN
28   62  55  95.0
29   64  96   NaN
30   65  83   NaN
31   66  89   NaN
32   68  79  69.0
33   70  74  84.0
34   72  93  66.0
35   76  97  58.0
36   77  97  89.0
37   78  89   NaN
38   79  81   NaN
39   80  93  79.0
40   81  64  92.0
41   82  92   NaN
42   83  80   NaN
43   86  57   NaN
44   88  73   NaN
45   91  68  53.0
46   93  91  62.0
47   95  75   NaN
48   96  61  97.0
49   98  93  67.0
50  100  98  59.0


In [5]:
#An inner join works in a similar way except it only returns records that are in both tables
query = """SELECT Class1.StudentID AS ID,
    Class1.Grade1 AS G1,
    Class2.Grade2 AS G2 FROM Class1
    INNER JOIN Class2 ON Class1.StudentID = Class2.StudentID;"""
df = pd.read_sql(query, conn)
print(df)

     ID  G1  G2
0     5  51  70
1    16  87  62
2    25  59  63
3    27  84  84
4    30  86  68
5    33  75  52
6    42  55  59
7    47  78  75
8    55  95  76
9    59  98  69
10   60  64  97
11   62  55  95
12   68  79  69
13   70  74  84
14   72  93  66
15   76  97  58
16   77  97  89
17   80  93  79
18   81  64  92
19   91  68  53
20   93  91  62
21   96  61  97
22   98  93  67
23  100  98  59


In [6]:
#Union combines two results, if we rename the columns to be ID and Grade for both, then we can get the following
query = """SELECT StudentID AS ID,
Grade1 AS Grade
FROM Class1
UNION
SELECT StudentID AS ID,
Grade2 AS Grade
FROM Class2;"""

df = pd.read_sql(query, conn)
print(df)

     ID  Grade
0     2     88
1     5     51
2     5     70
3     7     83
4     9     86
5    10     94
6    11     88
7    14     54
8    15     51
9    16     62
10   16     87
11   17     66
12   23     63
13   25     59
14   25     63
15   27     84
16   28     84
17   30     68
18   30     86
19   31     62
20   32     89
21   33     52
22   33     75
23   34     84
24   35     60
25   40     63
26   41     93
27   42     55
28   42     59
29   43     72
..  ...    ...
63   76     58
64   76     97
65   77     89
66   77     97
67   78     89
68   79     81
69   80     79
70   80     93
71   81     64
72   81     92
73   82     92
74   83     80
75   84     78
76   86     57
77   88     73
78   90     69
79   91     53
80   91     68
81   92     71
82   93     62
83   93     91
84   95     75
85   96     61
86   96     97
87   97     67
88   98     67
89   98     93
90   99     50
91  100     59
92  100     98

[93 rows x 2 columns]


In [7]:
#Of course we might want a way to denote the classroom, we can give the constants 1 and 2 for class
query = """SELECT StudentID AS ID,
Grade1 AS Grade,
1 AS Class
FROM Class1
UNION
SELECT StudentID AS ID,
Grade2 AS Grade,
2 AS Class
FROM Class2;"""

df = pd.read_sql(query, conn)
print(df)

     ID  Grade  Class
0     2     88      2
1     5     51      1
2     5     70      2
3     7     83      1
4     9     86      2
5    10     94      1
6    11     88      2
7    14     54      2
8    15     51      1
9    16     62      2
10   16     87      1
11   17     66      1
12   23     63      1
13   25     59      1
14   25     63      2
15   27     84      1
16   27     84      2
17   28     84      2
18   30     68      2
19   30     86      1
20   31     62      1
21   32     89      2
22   33     52      2
23   33     75      1
24   34     84      2
25   35     60      1
26   40     63      2
27   41     93      1
28   42     55      1
29   42     59      2
..  ...    ...    ...
64   76     58      2
65   76     97      1
66   77     89      2
67   77     97      1
68   78     89      1
69   79     81      1
70   80     79      2
71   80     93      1
72   81     64      1
73   81     92      2
74   82     92      1
75   83     80      1
76   84     78      2
77   86   

In [8]:
#ORDER BY will sort the table by a given column
query = """SELECT * FROM Class1 ORDER BY Grade1;"""

df = pd.read_sql(query, conn)
print(df)

    StudentID  Grade1  Absent1
0           5      51        4
1          15      51        1
2          56      54        1
3          42      55        4
4          62      55        1
5          44      56        0
6          49      57        4
7          86      57        3
8          25      59        3
9          48      59        4
10         35      60        2
11         96      61        1
12         31      62        1
13         61      62        1
14         23      63        2
15         60      64        4
16         81      64        0
17         17      66        0
18         91      68        1
19         88      73        4
20         50      74        4
21         70      74        1
22         33      75        1
23         95      75        2
24         47      78        4
25         58      79        0
26         68      79        1
27         83      80        0
28         79      81        4
29          7      83        1
30         65      83        1
31      

In [9]:
#Passing the keyword ASC/DESC will change the sorting to either be ascending or descending
query = """SELECT * FROM Class1 ORDER BY Grade1 DESC;"""

df = pd.read_sql(query, conn)
print(df)

    StudentID  Grade1  Absent1
0          59      98        3
1         100      98        2
2          52      97        4
3          76      97        3
4          77      97        3
5          64      96        3
6          55      95        4
7          10      94        0
8          41      93        1
9          72      93        4
10         80      93        2
11         98      93        2
12         82      92        1
13         93      91        4
14         66      89        0
15         78      89        4
16         16      87        1
17         54      87        0
18         30      86        1
19         27      84        4
20          7      83        1
21         65      83        1
22         79      81        4
23         83      80        0
24         58      79        0
25         68      79        1
26         47      78        4
27         33      75        1
28         95      75        2
29         50      74        4
30         70      74        1
31      

In [10]:
#You can also sort by first one column then another
query = """SELECT * FROM Class1 ORDER BY Grade1, Absent1;"""

df = pd.read_sql(query, conn)
print(df)

    StudentID  Grade1  Absent1
0          15      51        1
1           5      51        4
2          56      54        1
3          62      55        1
4          42      55        4
5          44      56        0
6          86      57        3
7          49      57        4
8          25      59        3
9          48      59        4
10         35      60        2
11         96      61        1
12         31      62        1
13         61      62        1
14         23      63        2
15         81      64        0
16         60      64        4
17         17      66        0
18         91      68        1
19         88      73        4
20         70      74        1
21         50      74        4
22         33      75        1
23         95      75        2
24         47      78        4
25         58      79        0
26         68      79        1
27         83      80        0
28         79      81        4
29          7      83        1
30         65      83        1
31      

In [11]:
#And as well you are allowed to pick the sorting direction for each
query = """SELECT * FROM Class1 ORDER BY Grade1 DESC, Absent1 ASC;"""

df = pd.read_sql(query, conn)
print(df)

    StudentID  Grade1  Absent1
0         100      98        2
1          59      98        3
2          76      97        3
3          77      97        3
4          52      97        4
5          64      96        3
6          55      95        4
7          10      94        0
8          41      93        1
9          80      93        2
10         98      93        2
11         72      93        4
12         82      92        1
13         93      91        4
14         66      89        0
15         78      89        4
16         54      87        0
17         16      87        1
18         30      86        1
19         27      84        4
20          7      83        1
21         65      83        1
22         79      81        4
23         83      80        0
24         58      79        0
25         68      79        1
26         47      78        4
27         33      75        1
28         95      75        2
29         70      74        1
30         50      74        4
31      

In [12]:
#If we left join from the second class and then filter to only the null values, we get values that
#are only present in table 2
query = """SELECT Class2.StudentID AS ID,
    Class1.Grade1 AS G1,
    Class2.Grade2 AS G2 FROM Class2
    LEFT JOIN Class1 ON Class1.StudentID = Class2.StudentID WHERE Class1.StudentID IS NULL;"""
df = pd.read_sql(query, conn)
print(df)

    ID    G1  G2
0    2  None  88
1    9  None  86
2   11  None  88
3   14  None  54
4   28  None  84
5   32  None  89
6   34  None  84
7   40  None  63
8   43  None  72
9   51  None  82
10  53  None  71
11  67  None  86
12  73  None  87
13  74  None  81
14  84  None  78
15  90  None  69
16  92  None  71
17  97  None  67
18  99  None  50


In [13]:
#With UNION we could add these on to replicate an outer join
query = """SELECT Class1.StudentID AS ID,
    Class1.Grade1 AS G1,
    Class2.Grade2 AS G2 FROM Class1
    LEFT JOIN Class2 ON Class1.StudentID = Class2.StudentID
UNION
SELECT Class2.StudentID AS ID,
    Class1.Grade1 AS G1,
    Class2.Grade2 AS G2 FROM Class2
    LEFT JOIN Class1 ON Class1.StudentID = Class2.StudentID WHERE Class1.StudentID IS NULL;"""
df = pd.read_sql(query, conn)
print(df)

     ID    G1    G2
0     2   NaN  88.0
1     5  51.0  70.0
2     7  83.0   NaN
3     9   NaN  86.0
4    10  94.0   NaN
5    11   NaN  88.0
6    14   NaN  54.0
7    15  51.0   NaN
8    16  87.0  62.0
9    17  66.0   NaN
10   23  63.0   NaN
11   25  59.0  63.0
12   27  84.0  84.0
13   28   NaN  84.0
14   30  86.0  68.0
15   31  62.0   NaN
16   32   NaN  89.0
17   33  75.0  52.0
18   34   NaN  84.0
19   35  60.0   NaN
20   40   NaN  63.0
21   41  93.0   NaN
22   42  55.0  59.0
23   43   NaN  72.0
24   44  56.0   NaN
25   47  78.0  75.0
26   48  59.0   NaN
27   49  57.0   NaN
28   50  74.0   NaN
29   51   NaN  82.0
..  ...   ...   ...
40   64  96.0   NaN
41   65  83.0   NaN
42   66  89.0   NaN
43   67   NaN  86.0
44   68  79.0  69.0
45   70  74.0  84.0
46   72  93.0  66.0
47   73   NaN  87.0
48   74   NaN  81.0
49   76  97.0  58.0
50   77  97.0  89.0
51   78  89.0   NaN
52   79  81.0   NaN
53   80  93.0  79.0
54   81  64.0  92.0
55   82  92.0   NaN
56   83  80.0   NaN
57   84   NaN  78.0


In [14]:
#Add some new data
G1 = pd.DataFrame([[1, "Midterm", 90],
             [1, "Final", 85]], columns=['ID', 'Assignment', 'Grade'])
G2 = pd.DataFrame([[1, "Midterm", 87],
             [1, "Final", 89]], columns=['ID', 'Assignment', 'Grade'])
G1.to_sql("Grades1", conn)
G2.to_sql("Grades2", conn)

#Try a left join the wrong way
query = """SELECT Grades1.ID,
Grades1.Assignment AS C1Assignment,
Grades2.Assignment AS C2Assignment,
Grades1.Grade AS C1Grade,
Grades2.Grade AS C2Grade
FROM Grades1
    LEFT JOIN Grades2 ON Grades1.ID = Grades2.ID;"""
df = pd.read_sql(query, conn)
print(df)

   ID C1Assignment C2Assignment  C1Grade  C2Grade
0   1      Midterm        Final       90       89
1   1      Midterm      Midterm       90       87
2   1        Final        Final       85       89
3   1        Final      Midterm       85       87


In [15]:
query = """SELECT Grades1.ID,
Grades1.Assignment AS C1Assignment,
Grades2.Assignment AS C2Assignment,
Grades1.Grade AS C1Grade,
Grades2.Grade AS C2Grade
FROM Grades1
    LEFT JOIN Grades2 ON Grades1.ID = Grades2.ID AND Grades1.Assignment=Grades2.Assignment;"""
df = pd.read_sql(query, conn)
print(df)

   ID C1Assignment C2Assignment  C1Grade  C2Grade
0   1      Midterm      Midterm       90       87
1   1        Final        Final       85       89


In [16]:
query = """SELECT Grades1.ID,
Grades1.Assignment AS Assignment,
Grades1.Grade AS C1Grade,
Grades2.Grade AS C2Grade
FROM Grades1
    LEFT JOIN Grades2 ON Grades1.ID = Grades2.ID AND Grades1.Assignment=Grades2.Assignment;"""
df = pd.read_sql(query, conn)
print(df)

   ID Assignment  C1Grade  C2Grade
0   1    Midterm       90       87
1   1      Final       85       89
